# Análise Preditiva de Churn para Academias

**Objetivo:** Construir um modelo de Machine Learning para prever a probabilidade de churn de clientes em uma academia, utilizando um dataset de exemplo (Telco Customer Churn) como base para a estrutura da análise.

## 1. Importação das Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score

# Configurações de visualização
sns.set_style('whitegrid')
%matplotlib inline

## 2. Carregamento dos Dados

Carregando o dataset de exemplo. Em um cenário real, substituir pelo dataset específico da academia.

In [ ]:
# Caminho para o arquivo CSV (ajuste se necessário)
data_path = '../data/raw/telco_churn_sample.csv'

try:
    df = pd.read_csv(data_path)
    print("Dataset carregado com sucesso!")
    print(f"Shape do dataset: {df.shape}")
except FileNotFoundError:
    print(f"Erro: Arquivo não encontrado em {data_path}")
    # Em um caso real, tratar o erro apropriadamente
    df = pd.DataFrame() # Cria um DataFrame vazio para evitar erros posteriores

In [ ]:
# Visualizar as primeiras linhas e informações básicas
if not df.empty:
    print("
Primeiras 5 linhas:")
    print(df.head())
    print("
Informações do Dataset:")
    print(df.info())
    print("
Estatísticas Descritivas:")
    print(df.describe(include='all'))

## 3. Análise Exploratória de Dados (EDA)

Nesta seção, exploramos os dados para entender melhor as distribuições, relações entre variáveis e identificar padrões iniciais relacionados ao churn.

In [ ]:
# Verificar valores ausentes
if not df.empty:
    print("
Valores Ausentes por Coluna:")
    print(df.isnull().sum())

In [ ]:
# Limpeza inicial (exemplo: converter TotalCharges para numérico, tratando erros)
if 'TotalCharges' in df.columns:
    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
    # Preencher valores ausentes em TotalCharges (ex: com a mediana ou média)
    if df['TotalCharges'].isnull().any():
        median_total_charges = df['TotalCharges'].median()
        df['TotalCharges'].fillna(median_total_charges, inplace=True)
        print(f"Valores ausentes em TotalCharges preenchidos com a mediana: {median_total_charges}")

In [ ]:
# Visualização da distribuição da variável alvo (Churn)
if 'Churn' in df.columns:
    plt.figure(figsize=(6, 4))
    sns.countplot(x='Churn', data=df)
    plt.title('Distribuição de Churn')
    plt.show()
    print(df['Churn'].value_counts(normalize=True))

*Adicionar mais visualizações aqui:*
    *   Histogramas para variáveis numéricas (ex: tenure, MonthlyCharges).
    *   Gráficos de barras para variáveis categóricas (ex: Contract, PaymentMethod).
    *   Boxplots para comparar distribuições numéricas por Churn.
    *   Análise de correlação entre variáveis numéricas.

## 4. Pré-processamento de Dados

Preparação dos dados para a modelagem, incluindo tratamento de variáveis categóricas e escalonamento.

In [ ]:
if not df.empty and 'Churn' in df.columns:
    # Remover colunas irrelevantes (ex: customerID)
    if 'customerID' in df.columns:
        df_processed = df.drop('customerID', axis=1)
    else:
        df_processed = df.copy()

    # Separar features (X) e variável alvo (y)
    X = df_processed.drop('Churn', axis=1)
    y = df_processed['Churn'].apply(lambda x: 1 if x == 'Yes' else 0) # Converter para binário

    # Identificar colunas numéricas e categóricas
    numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
    categorical_features = X.select_dtypes(include=['object']).columns

    print(f"Features Numéricas: {list(numeric_features)}")
    print(f"Features Categóricas: {list(categorical_features)}")

In [ ]:
# Criar transformadores para o pipeline
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Criar o pré-processador com ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

## 5. Divisão dos Dados (Treino e Teste)

In [ ]:
if not df.empty and 'Churn' in df.columns:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    print(f"Tamanho do conjunto de treino: {X_train.shape}, {y_train.shape}")
    print(f"Tamanho do conjunto de teste: {X_test.shape}, {y_test.shape}")

## 6. Treinamento do Modelo

Treinaremos alguns modelos de classificação usando Pipelines para incluir o pré-processamento.

In [ ]:
# Modelo 1: Regressão Logística
if not df.empty and 'Churn' in df.columns:
    pipeline_lr = Pipeline(steps=[('preprocessor', preprocessor),
                                ('classifier', LogisticRegression(random_state=42, max_iter=1000))])

    pipeline_lr.fit(X_train, y_train)
    print("Modelo de Regressão Logística treinado.")

In [ ]:
# Modelo 2: Random Forest
if not df.empty and 'Churn' in df.columns:
    pipeline_rf = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', RandomForestClassifier(random_state=42))])

    pipeline_rf.fit(X_train, y_train)
    print("Modelo Random Forest treinado.")

*Adicionar outros modelos se desejar (ex: Gradient Boosting, SVM).*

## 7. Avaliação dos Modelos

Avaliamos o desempenho dos modelos no conjunto de teste.

In [ ]:
def evaluate_model(model, X_test, y_test, model_name):
    """Função para avaliar um modelo e imprimir métricas."""
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1] # Probabilidade da classe positiva

    print(f"--- Avaliação: {model_name} ---")
    print(f"Acurácia: {accuracy_score(y_test, y_pred):.4f}")
    print(f"AUC-ROC: {roc_auc_score(y_test, y_proba):.4f}")
    print("
Relatório de Classificação:")
    print(classification_report(y_test, y_pred))
    print("
Matriz de Confusão:")
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Previsto')
    plt.ylabel('Verdadeiro')
    plt.title(f'Matriz de Confusão - {model_name}')
    plt.show()
    print("-------------------------------------")

In [ ]:
# Avaliar Regressão Logística
if not df.empty and 'Churn' in df.columns:
    evaluate_model(pipeline_lr, X_test, y_test, "Regressão Logística")

In [ ]:
# Avaliar Random Forest
if not df.empty and 'Churn' in df.columns:
    evaluate_model(pipeline_rf, X_test, y_test, "Random Forest")

## 8. Conclusão e Próximos Passos

Resumo dos resultados e sugestões para trabalhos futuros.

*   Comparar os modelos e selecionar o melhor com base nas métricas relevantes (ex: AUC-ROC, Recall para a classe 'Churn').
*   Realizar ajuste fino de hiperparâmetros (Hyperparameter Tuning) no melhor modelo.
*   Analisar a importância das features (Feature Importance) para entender os drivers do churn.
*   Interpretar o modelo (ex: usando SHAP ou LIME).
*   Considerar técnicas para lidar com desbalanceamento de classe (ex: SMOTE, ajuste de pesos).
*   Implantar o modelo final (ex: via API).